In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import os
os.environ['HF_HOME'] = './cache'
print("huggingface cache is in {}".format(os.getenv('HF_HOME')))

huggingface cache is in ./cache


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_model = 'NousResearch/Llama-2-7b-chat-hf'
# Fine-tuned model
# new_model = "./security_miles_model/filter_sec"
new_model = "./security_miles_model/ppo_model"
# log_dir = "results/filter_sec/runs"
# log_dir = "results/reward_model/runs"
log_dir = "results/ppo_model/trl"

In [ ]:
from tensorboard import notebook
notebook.start("--logdir {} --port 4004".format(log_dir))

In [5]:
# Reload model in FP16 and merge it with LoRA weights
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    # some paramerters from https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035/3
    temperature=1.0,
    # do_sample=True
)
model = PeftModel.from_pretrained(load_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|████████████████████████████████████████████████| 2/2 [01:01<00:00, 30.93s/it]
/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the correspond

In [6]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [7]:
# Run text generation pipeline with our next model
prompt = "Who is Leonardo Da Vinci?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man - an Italian polymath, artist, inventor, engineer, and scientist. He is widely considered one of the most diversely talented individuals to have ever lived.

Da Vinci was born in Vinci, Italy, and was the illegitimate son of a notary named Ser Piero and a peasant woman named Caterina Buti. Despite his humble beginnings, he rose to become one of the most renowned artists and thinkers of the Renaissance.

Da Vinci's artistic career began early, and he was apprenticed to the artist Andrea del Verrocchio in Florence. There, he honed his skills in painting, sculpture, and engineering. Some of his most famous works include the Mona Lisa, The


In [8]:
prompt = "What is result of 1+1?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is result of 1+1? [/INST]  The result of 1 + 1 is 2.


In [9]:
prompt = "What's shellcode? and what's relative addressing technique when you want to implement spwning a shell?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What's shellcode? and what's relative addressing technique when you want to implement spwning a shell? [/INST]  Shellcode is a term used in the context of exploitation and reverse engineering. It refers to the code that is executed by a vulnerable program or operating system after a successful exploitation, allowing an attacker to take control of the system or perform malicious actions.

In the context of implementing a shell, relative addressing technique is a method of calculating the address of a memory location relative to the address of a base pointer. This technique is commonly used in shellcode to calculate the address of a specific location in memory, such as the address of a system call or the address of a buffer that contains shellcode.

Here's an example of how relative addressing technique can be used to implement a shell:

Suppose you want to create a shell that can execute arbitrary commands on a vulner


In [10]:
prompt = "what is a hacker?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] what is a hacker? [/INST]  The term "hacker" has multiple meanings and connotations, and can be used in different contexts. Here are some possible definitions:

1. Computer security expert: A hacker is someone who specializes in computer security and uses their skills to identify vulnerabilities and weaknesses in computer systems, networks, and applications. Ethical hackers, also known as white-hat hackers, use their skills to help organizations improve their security and protect against cyber threats.
2. Malicious cybercriminal: A hacker can also refer to someone who uses their computer skills to engage in malicious activities, such as hacking into computer systems or networks without authorization, stealing sensitive information, or disrupting operations. These hackers are often referred to as black-hat hackers.
3. Computer enthusiast: The term "hacker" can also be


In [12]:
prompt = "Can you use GPG to decrypt anbd encrypt with a different key in one step?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Can you use GPG to decrypt anbd encrypt with a different key in one step? [/INST]  Yes, you can use GPG to decrypt and encrypt with a different key in one step by using the `--revoke` and `--encrypt` options.

Here's an example of how you can use GPG to decrypt and encrypt a message with a different key in one step:
```
gpg --revoke <old-key-id> --encrypt <message> --key <new-key-id>
```
In this command, replace `<old-key-id>` with the ID of the key you want to revoke, `<message>` with the message you want to encrypt, and `<new-key-id>` with the ID of the key you want to use for encryption.

When you run this command, GPG will first revoke the old key and then encrypt the


In [13]:
prompt = "How to securely store secrets in a Docker container?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How to securely store secrets in a Docker container? [/INST]  Storing secrets securely in a Docker container is important to protect sensitive information from unauthorized access. Here are some best practices for securingly storing secrets in a Docker container:

1. Use environment variables: Store secrets as environment variables in the Docker container. This way, the secrets are not stored in the container's file system, which is accessible to the user running the container. Instead, the secrets are stored in memory, which is more secure.
2. Use a secure storage solution: Use a secure storage solution, such as a secret management tool, to store the secrets. These tools provide a secure way to store and manage sensitive information, such as passwords, API keys, and other secrets. Some popular secret management tools include Hashicorp's Vault, AWS Secrets Manager, and Google Cloud Secret Manager


## simple SFT

In [11]:
# Run text generation pipeline with our next model
prompt = "Who is Leonardo Da Vinci?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid($userid); $group = $db-&gt;get_groupbyid($groupid); $members = $group-&gt;get_members(); foreach($members as $key =&gt; $value) { echo $value-&gt;get_id(); } i hope this helps. you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid($userid); $group = $db-&gt;get_groupbyid($groupid); $members = $group-&gt;get_members


In [12]:
prompt = "What is result of 1+1?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is result of 1+1? [/INST] you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid($userid); $group = $db-&gt;get_groupbyid($groupid); $members = $group-&gt;get_members(); foreach($members as $key =&gt; $value) { echo $value-&gt;get_name(); } i hope this helps. you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid($userid); $group = $db-&gt;get_groupbyid($groupid); $members = $group-&gt;get_


In [13]:
prompt = "what is a hacker?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] what is a hacker? [/INST] i've been using the google maps api for a while now, and i've found that the best way to do this is to use the geocoder class. it's a bit of a pain to get the coordinates right, but once you do, it's easy to get the lat/lng of any address. here's a quick example: http://code.google.com/p/google-maps-api-samples/source/browse/samples/address_to_latlng.php i've also found that the google maps api is a lot easier to use than the openstreetmap api. i've never used the openstreetmap api, but i've heard it's a lot more complicated to use than the google maps api. i've been using the google maps api for a while now, and i've found that the best


In [14]:
prompt = "How to securely store secrets in a Docker container?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How to securely store secrets in a Docker container? [/INST] you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid($userid); $group = $db-&gt;get_groupbyid($groupid); $members = $group-&gt;get_members(); foreach($members as $key =&gt; $value) { if($value-&gt;get_id() == $user-&gt;get_id()) { echo "you are in the same group as the user who is logged in."; } } i hope this helps. you can use the following code to get the list of all the users who are in the same group as the user who is logged in. $user = $db-&gt;get_userbyid


In [15]:
prompt = "What's shellcode? and what's relative addressing technique when you want to implement spwning a shell?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What's shellcode? and what's relative addressing technique when you want to implement spwning a shell? [/INST] you can use the following code to get the list of all the users in your project. you can then use the user.get() method to get the details of each user. list&lt;user&gt; userlist = new arraylist&lt;&gt;(); for(user user : user.list("projectid")) { userlist.add(user); } you can then use the userlist to get the details of each user. for example, you can use the following code to get the details of a user. user user = userlist.get(0); system.out.println(user.get().getname()); you can also use the user.get() method to get the details of a user. system.out.println(user.get().getname()); you can also use
